# Sistema Massa-Mola

### Fundamento Teórico:

O movimento de um sistema massa-mola não amortecido é descrito pela seguinte equação diferencial matricial:

$$ [M] \ddot{\vec{x}} + [K] \vec{x} = \vec{0} $$

Onde:
- $[M]$ é a **matriz de massa** (geralmente diagonal).
- $[K]$ é a **matriz de rigidez**.
- $\vec{x}$ é o vetor de deslocamentos das massas.
- $\ddot{\vec{x}}$ é o vetor de acelerações das massas.

Para encontrar as frequências naturais, se assume uma solução harmônica da forma $\vec{x}(t) = \vec{v} e^{i\omega t}$, onde $\vec{v}$ é o vetor de amplitudes (o modo normal) e $\omega$ é a frequência natural. Substituindo na equação, obtém-se o **problema de autovalor generalizado**:

$$ ([K] - \omega^2 [M]) \vec{v} = \vec{0} $$

cujas soluções não triviais ($\vec{v} \neq \vec{0}$) existem somente se o determinante da matriz for nulo. A solução fornece os autovalores $\lambda = \omega^2$ e os autovetores correspondentes $\vec{v}$ (modos normais). De forma que as frequências naturais são, portanto, $\omega = \sqrt{\lambda}$.

In [4]:
import base64
from IPython.display import Image, display

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

In [11]:
diagrama = """
flowchart TD
    A([Início]) --> B["Definição das Matrizes<br>de Massa e Rigidez"];
    B --> C["Resolução do Sistema<br>(resolver_sistema)"];
    C --> D["Cálculo de Autovalores<br>e Autovetores"];
    D --> E["Obtenção das Frequências<br>Naturais e Modos Normais"];
    
    subgraph Visualizações
        direction TB
        E --> F["Plotagem dos Modos<br>Normais de Vibração<br>(plotar_modos_plotly)"];
        E --> G["Análise de Amplitudes<br>Relativas por Massa<br>(plotar_amplitude_modos_plotly)"];
        E --> H["Frequências vs<br>Variação de Massa<br>(plotar_frequencias_vs_massa)"];
        E --> I["Frequências vs<br>Variação de Rigidez<br>(plotar_frequencias_vs_rigidez)"];
    end
    
    subgraph Saídas
        direction LR
        J["Gráficos Interativos<br>dos Modos"];
        K["Análises Paramétricas<br>Dinâmicas"];
        L["Resultados Numéricos<br>Impressos"];
    end
    
    F --> J;
    G --> J;
    H --> K;
    I --> K;
    E --> L;
    
    J --> Z([Fim]);
    K --> Z;
    L --> Z;
"""

mm(diagrama)

In [6]:
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.express as px

np.set_printoptions(precision=4, suppress=True)

def resolver_sistema(M, K):
    """
    Resolve o problema de autovalor para encontrar as frequências e modos normais.
    
    Argumentos:
    M (np.array): Matriz de massa
    K (np.array): Matriz de rigidez
    
    Retorna:
    frequencias (np.array): Vetor com as frequências naturais (rad/s)
    modos_normais (np.array): Matriz onde cada coluna é um modo normal de vibração
    """
    M_inv = np.linalg.inv(M)
    A = M_inv @ K
    
    autovalores, autovetores = np.linalg.eig(A)
    
    indices_ordenados = np.argsort(autovalores)
    autovalores_ordenados = autovalores[indices_ordenados]
    modos_normais_ordenados = autovetores[:, indices_ordenados]
    
    frequencias_naturais = np.sqrt(autovalores_ordenados)
    
    return frequencias_naturais, modos_normais_ordenados

def plotar_modos(frequencias, modos_normais, titulo):
    """Plotar os modos normais de vibração usando Plotly."""
    num_massas = modos_normais.shape[0]
    num_modos = modos_normais.shape[1]
    posicoes_massas = np.arange(1, num_massas + 1)
    
    fig = make_subplots(
        rows=1, cols=num_modos,
        subplot_titles=[f"Modo {i+1}<br>ω = {frequencias[i]:.4f} rad/s" for i in range(num_modos)],
        shared_yaxes=True
    )
    
    for i in range(num_modos):
        modo = modos_normais[:, i]
        modo_normalizado = modo / np.max(np.abs(modo))
        
        x_plot = np.concatenate(([0], posicoes_massas, [num_massas + 1]))
        y_plot = np.concatenate(([0], modo_normalizado, [0]))
        
        fig.add_trace(
            go.Scatter(
                x=x_plot, y=y_plot,
                mode='lines+markers',
                line=dict(color='dodgerblue', width=2, dash='dash'),
                marker=dict(color='red', size=10),
                name=f'Modo {i+1}',
                showlegend=False
            ),
            row=1, col=i+1
        )
        
        fig.add_hline(y=0, line_dash="dot", line_color="black", opacity=0.5, row=1, col=i+1)
        
        fig.update_xaxes(
            title_text="Posição",
            tickvals=x_plot,
            ticktext=['Parede'] + [f'm{j}' for j in range(1, num_massas + 1)] + ['Parede'],
            row=1, col=i+1
        )
        
        if i == 0:
            fig.update_yaxes(title_text="Deslocamento Relativo", row=1, col=1)
    
    fig.update_layout(
        title=f"Modos Normais de Vibração - {titulo}",
        height=400,
        showlegend=False
    )
    
    fig.show()

def plotar_amplitude_modos(frequencias, modos_normais, titulo):
    """Plotar a amplitude relativa de cada massa para todos os modos."""
    num_massas = modos_normais.shape[0]
    num_modos = modos_normais.shape[1]
    
    modos_normalizados = np.zeros_like(modos_normais)
    for i in range(num_modos):
        modos_normalizados[:, i] = modos_normais[:, i] / np.max(np.abs(modos_normais[:, i]))
    
    fig = go.Figure()
    
    cores = px.colors.qualitative.Set1
    massas_labels = [f'Massa {i+1}' for i in range(num_massas)]
    
    for i in range(num_modos):
        fig.add_trace(go.Bar(
            x=massas_labels,
            y=modos_normalizados[:, i],
            name=f'Modo {i+1} (ω={frequencias[i]:.3f})',
            marker_color=cores[i % len(cores)],
            opacity=0.8
        ))
    
    fig.update_layout(
        title=f"Amplitude Relativa dos Modos Normais - {titulo}",
        xaxis_title="Massas",
        yaxis_title="Amplitude Relativa",
        barmode='group',
        height=500,
        hovermode='x unified'
    )
    
    fig.show()

def imprimir_e_plotar_resultados(frequencias, modos_normais, titulo):
    print("-" * 60)
    print(titulo)
    print("-" * 60)
    for i in range(len(frequencias)):
        print(f"Frequência Natural {i+1} (ω_{i+1}): {frequencias[i]:.4f} rad/s")
        modo_normalizado = modos_normais[:, i] / np.max(np.abs(modos_normais[:, i]))
        print(f"Modo Normal {i+1} (deslocamentos relativos): {modo_normalizado}\n")
    
    plotar_modos(frequencias, modos_normais, titulo)

    plotar_amplitude_modos(frequencias, modos_normais, titulo)

def plotar_frequencias_vs_massa(M_base, K_base, indice_massa=0, titulo_massa="m₁", 
                                      massas_variacao=np.linspace(0.5, 5, 20)):
    """Plotar frequências vs variação de massa."""
    frequencias_resultados = []
    
    for m in massas_variacao:
        M_temp = M_base.copy()
        M_temp[indice_massa, indice_massa] = m
        
        freq, _ = resolver_sistema(M_temp, K_base)
        frequencias_resultados.append(freq)
    
    frequencias_resultados = np.array(frequencias_resultados)
    num_modos = frequencias_resultados.shape[1]
    
    fig = go.Figure()
    cores = px.colors.qualitative.Set1
    
    for i in range(num_modos):
        fig.add_trace(go.Scatter(
            x=massas_variacao,
            y=frequencias_resultados[:, i],
            mode='lines+markers',
            name=f'Modo {i+1}',
            line=dict(color=cores[i % len(cores)], width=3),
            marker=dict(size=6)
        ))
    
    fig.update_layout(
        title=f"Frequências Naturais vs Variação de Massa ({titulo_massa})",
        xaxis_title=f"Massa {titulo_massa} (kg)",
        yaxis_title="Frequência Natural (rad/s)",
        height=500,
        hovermode='x unified'
    )
    
    fig.show()

def plotar_frequencias_vs_rigidez(M_base, K_base, fator_escala=True, 
                                       rigidez_variacao=np.linspace(0.5, 3, 20)):
    """Plotar frequências vs variação de rigidez."""
    frequencias_resultados = []
    K_original = K_base.copy()
    
    for escala in rigidez_variacao:
        if fator_escala:
            K_temp = K_original * escala
        else:
            K_temp = K_original + escala
        
        freq, _ = resolver_sistema(M_base, K_temp)
        frequencias_resultados.append(freq)
    
    frequencias_resultados = np.array(frequencias_resultados)
    num_modos = frequencias_resultados.shape[1]
    
    fig = go.Figure()
    cores = px.colors.qualitative.Set1
    
    for i in range(num_modos):
        fig.add_trace(go.Scatter(
            x=rigidez_variacao,
            y=frequencias_resultados[:, i],
            mode='lines+markers',
            name=f'Modo {i+1}',
            line=dict(color=cores[i % len(cores)], width=3),
            marker=dict(size=6)
        ))
    
    titulo_x = "Fator de Escala da Rigidez" if fator_escala else "Incremento de Rigidez (N/m)"
    
    fig.update_layout(
        title="Frequências Naturais vs Variação de Rigidez",
        xaxis_title=titulo_x,
        yaxis_title="Frequência Natural (rad/s)",
        height=500,
        hovermode='x unified'
    )
    
    fig.show()

## 1. Sistema com Duas Massas e Extremos Fixos

Vamos modelar o sistema abaixo:

`Parede --- mola(k1) --- [m1] --- mola(k2) --- [m2] --- mola(k3) --- Parede`

A matriz de massa $[M]$ e a matriz de rigidez $[K]$ são:

$$
[M] = \begin{bmatrix} m_1 & 0 \\ 0 & m_2 \end{bmatrix}
\quad
[K] = \begin{bmatrix} k_1+k_2 & -k_2 \\ -k_2 & k_2+k_3 \end{bmatrix}
$$

#### 1.1. Caso Base: Massas e Molas Iguais

Inicialmente, `m1 = m2 = 1 kg` e `k1 = k2 = k3 = 1 N/m`.

In [12]:
# 1.1. Sistema de 2 massas - Caso Base
m1, m2 = 1.0, 1.0
k1, k2, k3 = 1.0, 1.0, 1.0

M_2massas = np.array([[m1, 0], [0, m2]])
K_2massas = np.array([[k1 + k2, -k2], [-k2, k2 + k3]])

frequencias, modos = resolver_sistema(M_2massas, K_2massas)
imprimir_e_plotar_resultados(frequencias, modos, "Sistema 2 Massas - Caso Base")

# Análises paramétricas
print("\n=== ANÁLISES PARAMÉTRICAS ===")
plotar_frequencias_vs_massa(M_2massas, K_2massas, indice_massa=1, titulo_massa="m₂")
plotar_frequencias_vs_rigidez(M_2massas, K_2massas)

------------------------------------------------------------
Sistema 2 Massas - Caso Base
------------------------------------------------------------
Frequência Natural 1 (ω_1): 1.0000 rad/s
Modo Normal 1 (deslocamentos relativos): [1. 1.]

Frequência Natural 2 (ω_2): 1.7321 rad/s
Modo Normal 2 (deslocamentos relativos): [ 1. -1.]




=== ANÁLISES PARAMÉTRICAS ===


#### 1.2. Variando a Massa `m2`

Agora, vamos manter `m1 = 1 kg` e `k = 1 N/m` fixos, mas variar o valor de `m2` para observar o impacto nas frequências e nos modos.

In [13]:
# 1.2. Sistema de 2 massas - Variando m2
m1 = 1.0
k1, k2, k3 = 1.0, 1.0, 1.0
valores_m2 = [2.0, 5.0, 10.0]

for m2_teste in valores_m2:
    M = np.array([[m1, 0], [0, m2_teste]])
    K = np.array([[k1 + k2, -k2], [-k2, k2 + k3]])
    
    frequencias, modos = resolver_sistema(M, K)
    titulo = f"Variação: m1=1, m2={m2_teste}, k=1"
    imprimir_e_plotar_resultados(frequencias, modos, titulo)

    # Análises paramétricas
    print("\n=== ANÁLISES PARAMÉTRICAS ===")
    plotar_frequencias_vs_massa(M, K, indice_massa=1, titulo_massa="m₂")
    plotar_frequencias_vs_rigidez(M, K)

------------------------------------------------------------
Variação: m1=1, m2=2.0, k=1
------------------------------------------------------------
Frequência Natural 1 (ω_1): 0.7962 rad/s
Modo Normal 1 (deslocamentos relativos): [0.7321 1.    ]

Frequência Natural 2 (ω_2): 1.5382 rad/s
Modo Normal 2 (deslocamentos relativos): [ 1.    -0.366]




=== ANÁLISES PARAMÉTRICAS ===


------------------------------------------------------------
Variação: m1=1, m2=5.0, k=1
------------------------------------------------------------
Frequência Natural 1 (ω_1): 0.5324 rad/s
Modo Normal 1 (deslocamentos relativos): [0.5826 1.    ]

Frequência Natural 2 (ω_2): 1.4548 rad/s
Modo Normal 2 (deslocamentos relativos): [ 1.     -0.1165]




=== ANÁLISES PARAMÉTRICAS ===


------------------------------------------------------------
Variação: m1=1, m2=10.0, k=1
------------------------------------------------------------
Frequência Natural 1 (ω_1): 0.3822 rad/s
Modo Normal 1 (deslocamentos relativos): [0.5394 1.    ]

Frequência Natural 2 (ω_2): 1.4332 rad/s
Modo Normal 2 (deslocamentos relativos): [ 1.     -0.0539]




=== ANÁLISES PARAMÉTRICAS ===


### **Análise dos Sistemas de Duas Massas**

O sistema de duas massas é o modelo fundamental para entender oscilações acopladas. Com dois graus de liberdade, ele exibe os principais conceitos de modos normais simétricos e anti-simétricos e como a quebra de simetria afeta o comportamento dinâmico.

#### **Análise do Caso Base (Massas Iguais $m_1 = m_2 = 1$)**

Configuração perfeitamente simétrica, onde os modos de vibração também exibem simetria.

* **Modos Normais:**
    * **Modo 1 (Simétrico, baixa frequência $\omega_1=1.0$ rad/s):** As duas massas se movem na mesma direção (em fase) e com a mesma amplitude. A mola central ($k_2$) é minimamente comprimida ou esticada, pois as massas se movem juntas. A frequência é mais baixa porque o sistema se comporta de forma análoga a um único bloco de massa maior ($m_1+m_2$) oscilando.
    * **Modo 2 (Anti-simétrico, alta frequência $\omega_2 \approx 1.732$ rad/s):** As duas massas se movem em direções opostas (fora de fase) com a mesma amplitude. Este movimento causa a máxima deformação da mola central, que armazena uma quantidade significativa de energia potencial. Esse comportamento aumenta a rigidez efetiva do sistema neste modo, resultando em uma frequência de vibração mais alta.

#### **Análise da Variação de $m_2$ (com $m_1$ fixo)**

Configuração que quebra a simetria da massa ($m_1 \neq m_2$) mudando drasticamente as frequências e as formas dos modos.

* **Frequências:**
    * Pela relação $\omega \propto 1/\sqrt{m}$, ambas as frequências naturais diminuem à medida que $m_2$ aumenta, pois o sistema se torna mais inercial.
    * O gráfico de frequências vs. massa $m_2$ revela um fenômeno de "cruzamento evitado" (*avoided crossing*). As curvas de frequência se aproximam, mas não se cruzam. Essa "repulsão" entre os níveis de energia (ou frequências) é característica de sistemas acoplados quando um parâmetro é variado.
* **Modos Normais e Comportamento Assintótico (quando $m_2 \gg m_1$):**
    * **Modo 1 (em fase):** À medida que $m_2$ aumenta, sua amplitude de movimento passa a dominar a do $m_1$. No limite de uma massa $m_2$ muito grande, a frequência $\omega_1$ tende a zero, pois seria necessário mover uma inércia quase infinita.
    * **Modo 2 (fora de fase):** Conforme $m_2$ se torna muito pesado, seu deslocamento relativo diminui drasticamente, se tornando um "nó" vibracional quase imóvel. O sistema se aproxima de um oscilador simples onde a massa leve $m_1$ vibra entre a parede (via $k_1$) e a "nova parede" formada por $m_2$ (via $k_2$). Por isso, a frequência $\omega_2$ não vai a zero, mas se aproxima de um valor constante.

## 2. Sistema com Três Massas e Extremos Fixos

O sistema agora é:

`Parede --- k1 --- [m1] --- k2 --- [m2] --- k3 --- [m3] --- k4 --- Parede`

As matrizes para este sistema são:

$$
[M] = \begin{bmatrix} m_1 & 0 & 0 \\ 0 & m_2 & 0 \\ 0 & 0 & m_3 \end{bmatrix}
\quad
[K] = \begin{bmatrix} k_1+k_2 & -k_2 & 0 \\ -k_2 & k_2+k_3 & -k_3 \\ 0 & -k_3 & k_3+k_4 \end{bmatrix}
$$

Vamos estudar diferentes distribuições de massa, mantendo `k=1 N/m` para todas as molas.

In [15]:
# 2. Sistema de 3 massas
k1, k2, k3, k4 = 1.0, 1.0, 1.0, 1.0
K = np.array([[k1 + k2, -k2, 0], [-k2, k2 + k3, -k3], [0, -k3, k3 + k4]])

distribuicoes_massa = {
    "Massas Iguais (1, 1, 1)": [1.0, 1.0, 1.0],
    "Massa Central Pesada (1, 5, 1)": [1.0, 5.0, 1.0],
    "Massas diferentes (2, 1, 3)": [2.0, 1.0, 3.0]
}

for nome_caso, massas in distribuicoes_massa.items():
    M = np.diag(massas)
    frequencias, modos = resolver_sistema(M, K)
    imprimir_e_plotar_resultados(frequencias, modos, nome_caso)
    
    # Análises paramétricas
    print(f"\n=== ANÁLISES PARAMÉTRICAS - {nome_caso} ===")
    plotar_frequencias_vs_massa(M, K, indice_massa=0, titulo_massa="m₁")
    plotar_frequencias_vs_rigidez(M, K)

------------------------------------------------------------
Massas Iguais (1, 1, 1)
------------------------------------------------------------
Frequência Natural 1 (ω_1): 0.7654 rad/s
Modo Normal 1 (deslocamentos relativos): [0.7071 1.     0.7071]

Frequência Natural 2 (ω_2): 1.4142 rad/s
Modo Normal 2 (deslocamentos relativos): [-1.  0.  1.]

Frequência Natural 3 (ω_3): 1.8478 rad/s
Modo Normal 3 (deslocamentos relativos): [-0.7071  1.     -0.7071]




=== ANÁLISES PARAMÉTRICAS - Massas Iguais (1, 1, 1) ===


------------------------------------------------------------
Massa Central Pesada (1, 5, 1)
------------------------------------------------------------
Frequência Natural 1 (ω_1): 0.4245 rad/s
Modo Normal 1 (deslocamentos relativos): [0.5495 1.     0.5495]

Frequência Natural 2 (ω_2): 1.4142 rad/s
Modo Normal 2 (deslocamentos relativos): [-1. -0.  1.]

Frequência Natural 3 (ω_3): 1.4899 rad/s
Modo Normal 3 (deslocamentos relativos): [ 1.     -0.2198  1.    ]




=== ANÁLISES PARAMÉTRICAS - Massa Central Pesada (1, 5, 1) ===


------------------------------------------------------------
Massas diferentes (2, 1, 3)
------------------------------------------------------------
Frequência Natural 1 (ω_1): 0.5626 rad/s
Modo Normal 1 (deslocamentos relativos): [0.7315 1.     0.952 ]

Frequência Natural 2 (ω_2): 0.9158 rad/s
Modo Normal 2 (deslocamentos relativos): [-1.     -0.3227  0.6253]

Frequência Natural 3 (ω_3): 1.5848 rad/s
Modo Normal 3 (deslocamentos relativos): [-0.3308  1.     -0.1807]




=== ANÁLISES PARAMÉTRICAS - Massas diferentes (2, 1, 3) ===


### **Análise dos Sistemas de Três Massas**

A introdução de uma terceira massa aumenta a complexidade do sistema, resultando em três frequências e modos normais distintos.

#### **Análise do Caso de Massas Iguais ($m_1 = m_2 = m_3 = 1$)**

Assim como no caso de duas masses, este é o mais simétrico, e essa simetria reflete-se diretamente nas formas dos modos de vibração.

* **Frequências:** O sistema possui três frequências naturais distintas e bem espaçadas.
* **Modos Normais:**
    * **Modo 1 (Fundamental, $\omega_1 \approx 0.765$ rad/s):** Todas as massas oscilam em fase (na mesma direção). A massa central ($m_2$) possui a maior amplitude, e as massas das extremidades ($m_1$, $m_3$) movem-se simetricamente com uma amplitude menor.
    * **Modo 2 (Intermediário, $\omega_2 \approx 1.414$ rad/s):** Este é um modo perfeitamente anti-simétrico. As massas externas ($m_1$, $m_3$) oscilam em oposição uma à outra com amplitudes iguais. A massa central ($m_2$) permanece completamente parada, agindo como um **nó vibracional**.
    * **Modo 3 (Superior, $\omega_3 \approx 1.848$ rad/s):** As massas externas ($m_1$, $m_3$) movem-se em fase uma com a outra, mas em oposição à massa central, que novamente possui a maior amplitude de movimento.

#### **Análise do Caso de Massa Central Pesada ($m_1 = 1$, $m_2 = 5$, $m_3 = 1$)**

A simetria espacial é mantida, mas a distribuição de massa deixa de ser não-uniforme.

* **Frequências:**
    * A frequência do **Modo 2 ($\omega_2 \approx 1.414$ rad/s)** permanece inalterada. Isso ocorre porque, neste modo, a massa central $m_2$ é um nó (não se move), então sua inércia não afeta essa vibração específica.
    * As frequências dos **Modos 1 e 3**, nos quais $m_2$ tem grande participação, diminuem significativamente, como esperado pelo aumento da inércia no sistema.
* **Modos Normais:**
    * **Modo 1:** A forma é similar ao caso simétrico, mas a amplitude da massa pesada $m_2$ é ainda mais dominante em relação às massas leves, que quase a acompanham como se estivessem "puxadas" por ela.
    * **Modo 2:** A forma é idêntica ao caso de massas iguais, com $m_2$ atuando como um nó.
    * **Modo 3:** A amplitude relativa da massa pesada $m_2$ diminui consideravelmente. É difícil oscilá-la em alta frequência, então as massas leves das extremidades passam a ter a maior amplitude de movimento.

#### **Análise do Caso de Massas Diferentes ($m_1 = 2$, $m_2 = 1$, $m_3 = 3$)**

Aqui há quebra completa da simetria, levando a modos de vibração mais complexos.

* **Frequências:** As frequências perdem o espaçamento regular do caso simétrico.
* **Modos Normais:**
    * Os modos perdem completamente a simetria visual. Não há mais amplitudes iguais ou nós perfeitamente no centro.
    * Ocorre o fenômeno de localização de modo, onde a energia vibracional de um modo específico tende a se concentrar em uma parte do sistema.
    * **Modo 1 (Fundamental):** Todas as massas ainda se movem na mesma direção geral, mas a massa mais leve ($m_2=1.0 kg$) exibe a maior amplitude, pois é a mais fácil de ser deslocada.
    * **Modos 2 e 3:** Apresentam padrões complexos de movimento em fase e fora de fase, onde as amplitudes relativas são ditadas pela interação entre a inércia de cada massa e a rigidez das molas que as conectam. Nota-se que no modo de maior frequência, a massa mais leve ($m_2$) novamente tem a maior amplitude relativa de movimento.